In [103]:
default_dict = Dict(
    :r => 10,
    :epsilon => 0.25,
    :kappa => 1.6,
    :delta => 1/3,
    :opacity => 0.65,
    :theta_frac => 1.0,
    :theta_cut => 0.0,
)

function tokamak_c(cur_dict=default_dict)
    term_0 = 0.3 * ( 1 - cur_dict[:delta] ^ 2 )
    term_1 = ( 1 - 2 * cur_dict[:delta] - term_0 ) / 8
    term_2 = cur_dict[:delta] / 2
    
    cur_c = [
        ( 0 - term_2 ),
        ( 1 + term_1 ),
        ( 0 + term_2 ),
        ( 0 - term_1 )
    ]
    
    cur_c
end

function tokamak_u(cur_phi, cur_dict=default_dict)
    cur_u = 0.0
    for (cur_index, cur_c) in enumerate(tokamak_c(cur_dict))
        cur_u += cur_c * cos( cur_phi * (cur_index-1) )
    end
    cur_u
end

tokamak_v(cur_phi, cur_dict=default_dict) = cur_dict[:kappa] * sin(cur_phi)

tokamak_r(cur_phi, cur_dict=default_dict) = cur_dict[:r] * ( 1 + cur_dict[:epsilon] * tokamak_u(cur_phi, cur_dict) )
tokamak_z(cur_phi, cur_dict=default_dict) = cur_dict[:r] * ( 0 + cur_dict[:epsilon] * tokamak_v(cur_phi, cur_dict) )

tokamak_x(cur_phi, cur_theta, cur_dict=default_dict) = tokamak_r(cur_phi, cur_dict) * cos(cur_theta)
tokamak_y(cur_phi, cur_theta, cur_dict=default_dict) = tokamak_r(cur_phi, cur_dict) * sin(cur_theta)

tokamak_z(cur_phi, cur_theta::AbstractFloat, cur_dict=default_dict) = tokamak_z(cur_phi, cur_dict)


tokamak_z (generic function with 4 methods)

In [87]:
using Plots
pgfplots()
cur_colors = get_color_palette(:auto, plot_color(:white), 17)

using DataStructures
return

In [116]:
cur_dicts = OrderedDict(
    "Tokamak" => deepcopy(default_dict),
    "Plasma" => deepcopy(default_dict)
)

cur_dicts["Tokamak"][:theta_frac] = 3/4
cur_dicts["Tokamak"][:theta_cut] = 1/4

cur_dicts["Plasma"][:epsilon] = 0.1
cur_dicts["Plasma"][:kappa] = 1.3
cur_dicts["Plasma"][:opacity] = 0.45

plot()

for (cur_index, (cur_key, tmp_dict)) in enumerate(cur_dicts)
    theta_count = 4 * (5) + 1
    phi_count = 4 * (3) + 1

    d_theta = 2 * pi / ( theta_count - 1 )
    d_phi = 2 * pi / ( phi_count - 1 )
    
    for cur_j in 1:Int( ( theta_count - 1 ) * tmp_dict[:theta_frac] )
        if iszero(tmp_dict[:theta_cut]) || tmp_dict[:theta_cut] < (cur_j/theta_count) < ( 1 - tmp_dict[:theta_cut] )
            cur_phi_list = 1:( phi_count - 1 )
        else
            cur_phi_list = ( Int( ( phi_count - 1 ) / 2 ) + 1 ):( phi_count - 1 ) 
        end

        for cur_i in cur_phi_list
            cur_xx = []
            cur_yy = []
            cur_zz = []

            prev_phi = ( cur_i - 1 ) * d_phi
            prev_theta = ( cur_j - 1 ) * d_theta

            next_phi = prev_phi + d_phi
            next_theta = prev_theta + d_theta        
            
            push!(cur_xx, tokamak_x(prev_phi, prev_theta, tmp_dict))
            push!(cur_yy, tokamak_y(prev_phi, prev_theta, tmp_dict))
            push!(cur_zz, tokamak_z(prev_phi, prev_theta, tmp_dict))
            push!(cur_xx, tokamak_x(prev_phi, next_theta, tmp_dict))
            push!(cur_yy, tokamak_y(prev_phi, next_theta, tmp_dict))
            push!(cur_zz, tokamak_z(prev_phi, next_theta, tmp_dict))
            push!(cur_xx, tokamak_x(next_phi, next_theta, tmp_dict))
            push!(cur_yy, tokamak_y(next_phi, next_theta, tmp_dict))
            push!(cur_zz, tokamak_z(next_phi, next_theta, tmp_dict))
            push!(cur_xx, tokamak_x(next_phi, prev_theta, tmp_dict))
            push!(cur_yy, tokamak_y(next_phi, prev_theta, tmp_dict))
            push!(cur_zz, tokamak_z(next_phi, prev_theta, tmp_dict))
            push!(cur_xx, tokamak_x(prev_phi, prev_theta, tmp_dict))
            push!(cur_yy, tokamak_y(prev_phi, prev_theta, tmp_dict))
            push!(cur_zz, tokamak_z(prev_phi, prev_theta, tmp_dict))

            plot!(cur_xx, cur_yy, cur_zz, fill=true, fillopacity=tmp_dict[:opacity], lineopacity=(2-cur_index), color=cur_colors[cur_index], width=0, label="")
        end
    end
end

plot!(aspectratio=:equal)

6
6
6
6
6
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 Z M 4.171875 0 "/>
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.109375 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 L 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 Z M 2.984375 -1.953125 L 0.5625 -1.953125 L 2.984375 -5.671875 Z M 2.984375 -1.953125 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.